# Targeting Direct Marketing with Amazon SageMaker XGBoost
_**Supervised Learning with Gradient Boosted Trees: A Binary Prediction Problem With Unbalanced Classes**_

---

---

## Contents

1. [背景](#背景)
1. [準備](#準備)
1. [Data](#Data)
    1. [Exploration](#Exploration)
    1. [Transformation](#Transformation)
1. [Training](#Training)
1. [Evaluation](#Evaluation)
1. [Additional：Amazon SageMaker Experiments & Amazon SageMaker Clarify](#Additional)
1. [Appendix1 - Feature Engineering](#Appendix1_Feature_Engineering)
1. [Appendix2 - Automatic Model Tuning(Optional)](#Appendix2_Automatic_Model_Tuning(Optional))
1. [Appendix3 - Serverless Deployment(Optional)](#Appendix3_Serverless_Deployment(Optional))

---

## 背景
ダイレクト・マーケティングは、郵便、電子メール、電話などを通じて、顧客を獲得するための一般的な戦術です。リソースや顧客の関心には限りがあるため、特定のオファーにエンゲージしそうな見込み客のサブセットのみをターゲットにすることが目標となります。 人口統計、過去の交流、環境要因のような容易に入手可能な情報に基づいてそれらの潜在顧客を予測することは、一般的な機械学習の問題です。

このノートブックでは、1回以上の電話の後、顧客が銀行で定期預金に加入するかどうかを予測する問題を扱います。 
手順は以下の通りです：
* Amazon SageMaker Studio Notebookの準備
* インターネットから Amazon SageMaker にデータをダウンロードする
* Amazon SageMaker のアルゴリズムに供給できるように、データを調査および変換する
* Gradient Boosting アルゴリズムを使用したモデルの学習・推論
* モデルの評価


---
## 準備

※ このノートブックは`ml.m4.xlarge`ノートブックインスタンスで作成され、テストされました。

まず、以下を指定しましょう：
- トレーニングデータとモデルデータに使用するS3バケットとプレフィックス。 これは、ノートブックインスタンス、トレーニング、ホスティングと同じリージョン内にある必要があります。
- trainingとhostingにデータへのアクセス権を与えるためのIAMロール。ロールの作成方法はドキュメントを参照してください。 ノートブックインスタンス、トレーニング、ホスティングに複数のロールが必要な場合は、boto正規表現を適切なIAMロールarn文字列で置き換えてください。

In [ ]:
# cell 02
import sagemaker
bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-dm'
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

Now let's bring in the Python libraries that we'll use throughout the analysis

In [ ]:
# cell 03
import sagemaker_datawrangler           # For interactive data prep widget
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker 
import zipfile     # Amazon SageMaker's Python SDK provides many helper functions

In [ ]:
# cell 04
pd.__version__

ここでpandasのバージョンが1.2.4以降になっていることを確認してください。そうでない場合は、先に進む前にカーネルを再起動してください。

---

## Data
サンプルデータセットとして [direct marketing dataset](https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip) をS3からダウンロードします。

\[Moro et al., 2014\] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014


In [ ]:
# cell 05
!wget https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip

with zipfile.ZipFile('bank-additional.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

では、これを Pandas データフレームに読み込んで見てみましょう。ここでは `sagemaker_datawrangler` ライブラリをインポートしたので、分布やデータの問題点、その他の有用な推奨事項や組み込みの変換を自動的に表示することができます。

In [ ]:
# cell 06
data = pd.read_csv('./bank-additional/bank-additional-full.csv')
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
data

データについて簡単に説明します。 Data Wranglerの出力からも分かるように、

* 顧客レコードは4万件強で、顧客ごとに20の特徴量があります。
* 特徴量は数値とカテゴリーが混在しています。
* データは少なくとも`時間`と`連絡先`でソートされているようです。

**各特徴の詳細:**

*人口統計*
* `age`： 顧客の年齢（数値）
* `job`： 仕事の種類 (カテゴリ: '管理'、'サービス'、...)
* `marital`： Marital 婚姻状況 (カテゴリー: '既婚', '独身', ...)
* `education`： 教育レベル (カテゴリー: 'basic.4y', 'high.school', ...)

*過去の顧客イベント*
* `default`： 債務不履行があるか（カテゴリカル：'no', 'unknown', ...）
* `housing`住宅ローンがあるか？住宅ローンがあるか？
* `loan`： 個人ローンを利用しているか（カテゴリー：'no', 'yes', ...）

*過去のダイレクトマーケティングコンタクト*
* `contact`： コンタクトの種類（カテゴリー：'cellular'、'telephone'、...）
* `月`： コンタクトの種類（カテゴリ： 'cellular', 'telephone', ...）
* `day_of_week`： 最終コンタクト曜日（カテゴリー：'mon'、'fri'、...）
* `duration`： 最終接触時間、秒単位（数値）。重要: duration = 0 の場合、`y` = 'no'となります。
 
*キャンペーン情報*
* `campaign` キャンペーン期間中にこのクライアントに対して行われたコンタクトの数 (数値、最後のコンタクトを含む)
* `pdays`： クライアントが前回のキャンペーンで最後にコンタクトしてから経過した日数 (数値)
* `previous`： このキャンペーンの前に、このクライアントに対して行ったコンタクトの数（数値）
* `poutcome`： 前回のマーケティングキャンペーンの結果（カテゴリー：'nonexistent'、'success'、...）

*外部環境要因*
* `emp.var.rate`： 雇用変動率 - 四半期指標 (数値)
* `cons.price.idx`： 消費者物価指数 - 月次指標 (数値)
* `cons.conf.idx`： 消費者信頼感指数 - 月次指標 (数値)
* `euribor3m`： Euribor の 3 か月物レート - 日次指標 (数値) 
* `nr.employed`： 雇用者数 - 四半期指標（数値）

*対象変数（Target変数）*
* `y`: 顧客が定期預金を契約しているか (バイナリ: 'yes', 'no')

### Exploration
Data Wrangler が表示しているデータフレームの結果をチェックしてみましょう。以下のようなことがわかります。

* 約 90% のターゲット変数 `y` が "no" であることがわかります。したがって、ほとんどの顧客は定期預金に加入していません。
* 説明変数となる多くの特徴量が "unknown "の値をとります。 何が "unknown "の値を引き起こしているのかは注意深く考える必要があります。
  * "unknown"が別カテゴリーとして含まれると判断したとしても、現実には、それらの観測量がその特徴量の他のカテゴリーに含まれる可能性が高いことを考えると、それは何を意味するのでしょうか？
* 説明変数となる多くの特徴量は、その中に非常に少数のオブザベーションを持つカテゴリを持つことがわかります。 もし小さなカテゴリーが、我々のターゲットとする結果を高度に予測することがわかったとして、それについて一般化するのに十分な証拠があるのでしょうか？
* コンタクトのタイミングは特に偏っています。 5月はほぼ3分の1、12月は1％未満。 これは来年12月の目標変数の予測にどのような意味を持つのでしょうか？
* 数値特徴には欠損値はないようです。 あるいは欠損値はすでにインプットされていると分かります。
  * `pdays`はほぼすべての顧客で1000に近い値を取ります。 おそらく、以前の接触がないことを示すPlaceholderの値でしょう。
* いくつかの数値特徴量には非常に長いテールがあります。 極端に大きな値を持つ少数のオブザベーションを異なる方法で処理する必要がありますか?
* いくつかの数値特徴量（特にマクロ経済的なもの）は、明確なバケットで発生します。 これらはcategoricalとして扱われるべきですか?



※ 本来の特徴量エンジニアリングでは様々な分析・試行錯誤を行うことがあります。例えば各特徴量間の相関関係を確認するなどです。

本ノートブックではそのステップを割愛します（セルの実行例は[Appendix](#Appendix)を参照）。（Data Wranglerを使用すればこういった分析もノーコードで行えます。）

### Transformation

生データをそのままデータ分析に利用するケースは少ないです。生データを加工してデータ分析に適した形に変換することはクレンジングなどと呼ばれますが、ほぼすべての機械学習プロジェクトにおいて必要なプロセスです。いくつかの一般的なテクニックが知られています。

* 欠損値の処理（欠損値を持つオブザベーションの除去、欠損値を持つ特徴の除去、欠損値のインプット、など）
* カテゴリ値から数値への変換
* 分布の正規化
* より複雑なデータタイプのCombination（画像、テキスト、等）

今回使用するのはXGBoostというアルゴリズムですが、これを含むいくつかのアルゴリズムは、疎なデータや奇妙に分布したデータをうまく処理する傾向があります。

ここでは、上の Exploration における分析などから、以下のように変換を加えると判断したとします。

In [ ]:
# cell 09

# Binarization
data['no_previous_contact'] = np.where(data['pdays'] == 999, 1, 0)                                 
data['not_working'] = np.where(np.in1d(data['job'], ['student', 'retired', 'unemployed']), 1, 0)   

# one-hot vectorize
model_data = pd.get_dummies(data)

# drop columns
model_data = model_data.drop(['duration', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'], axis=1)

それではデータセットを機械学習モデルで学習させるために分割します。
* `train_data`：学習用データセット。モデルの学習に使用するデータ。
* `validation+data`：バリデーションデータ。モデルの学習の際に、直接学習そのものには使用せず、適切に進行しているかどうかを判断するために使用するデータ。
* `test_data`：テストデータ。学習完了したモデルの性能を評価するためのデータ。

In [ ]:
# cell 12
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])

train_df = pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1)
valid_df = pd.concat([validation_data['y_yes'], validation_data.drop(['y_no', 'y_yes'], axis=1)], axis=1)

train_df.to_csv('train.csv', index=False, header=False)
valid_df.to_csv('validation.csv', index=False, header=False)

学習に使用するデータをAmazon S3にアップロードします。

In [ ]:
# cell 14
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

---

## Training
それでは実際にモデルの学習を行なっていきましょう。本ノートブックではXGBoostをアルゴリズムとして選択します。

`xgboost`は、勾配ブースティング木のための非常に人気のあるオープンソースパッケージです。`xgboost`は計算能力が高く、機能が充実しており、多くの機械学習コンテストで使用されています。 

`xgboost`を含むよく用いられるアルゴリズムは、Amazon SageMaker側が[組み込みアルゴリズム](https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html)を提供しています。こちらを指定することで、「学習自体のコード（train.pyなどのスクリプト）を書かなくても、データを指定することで学習する」ことができます。

まず、Amazon SageMaker の XGBoost 実装用の ECR コンテナの場所を指定する必要があります。

In [ ]:
# cell 15
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

次に、CSVファイルを使用してトレーニングするため、学習データがAmazon S3のどこにあるのかを表すポインタとして機能する変数を定義します。

In [ ]:
# cell 16
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

それでは実際にモデルを学習していきましょう。以下の流れで実行します。
1. `Estimator`を作成します。これはモデルの学習や推論に用いられるインターフェースです。この`Estimator`に学習パラメータを指定します。これには以下が含まれます。
    * `xgboost`アルゴリズムのコンテナ
    * 使用するIAMロール
    * トレーニングインスタンスの種類と数
    * 出力データのS3ロケーション
    * アルゴリズムのハイパーパラメータ
1. そして、`.fit()`関数で以下を指定します：
    * 入力となるデータのS3ロケーション。 今回の場合、訓練セットと検証セットの両方が渡される。

In [ ]:
# cell 17
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

---
## Evaluation
学習が完了したらモデルを評価してみましょう。

まずはモデルをデプロイします。デプロイすることで、入力データが与えられた際にリアルタイムで推論結果を返すことができるようになります。モデルのデプロイには、先ほど作成したモデルに`.deploy()`とするだけです。

In [ ]:
# cell 18
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

モデルのデプロイが完了したら、テストデータを推論してみます。

In [ ]:
# cell 19
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [ ]:
# cell 20
def predict(data, predictor, rows=500 ):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.drop(['y_no', 'y_yes'], axis=1).to_numpy(), xgb_predictor)

最後に、混同行列をチェックし、予測と実際の値がどの程度一致したかを確認してみましょう。

In [ ]:
# cell 21
pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

つまり、～4000人の潜在顧客のうち、約150人が購読すると予測し、そのうち約100人が実際に購読したとわかります。 一方で予測できなかった389人の購読者がいたことがわかります。 これはあまり望ましいことではありませんが、モデルはこれを改善するようにチューニングすることができます。最も重要なことは、最小限の努力で、我々のモデルは[ここ](https://core.ac.uk/download/pdf/55631291.pdf)で発表されたものと同様の精度を出したということです。

Amazon SageMakerではモデルのハイパーパラメータを自動でチューニングする機能が備わっています。このような機能を使用してモデルのチューニングを実施し、モデルの性能を高めることが可能です。（[Appendix2 - Automatic Model Tuning(Optional)](#Appendix2_Automatic_Model_Tuning(Optional)) 参照）

### Clean Up
このノートブックで作成したリソースのうち、不要となったデプロイしたモデルを削除します。

In [ ]:
# cell 22
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

---
## Additional
## Amazon SageMaker Experiments & Amazon SageMaker Clarify

Amazon SageMaker上でモデルの開発・試行錯誤をするのに便利な機能を紹介します。

* **Amazon SageMaker Experiments：実験管理機能**
    * 一般に、モデルの開発フェーズでは、アルゴリズムやモデルの構造、ハイパーパラメータなどの各種条件を変更して学習・評価を行う、様々な試行錯誤を伴います。その際に、モデルの学習条件やその評価指標、出力となるモデルのパスなどを紐づけて一箇所で管理できる機能です。
    * 複数の試行錯誤の結果を比較して可視化したり、分析することが可能です。 
* **Amazon SageMaker Clarify：モデルの説明性・バイアスの分析**
    * 機械学習のバイアスを理解するために、MLの各プロセスにおけるデータ・モデルの不均衡の検出を行います。
      * データ準備時のバイアスの検出
      * 訓練されたモデルのバイアスを検出
      * 特徴量分析 - どの特徴量が予測に影響を与えたか（寄与率）
      * 分析結果のレポートの提供

### Amazon SageMaker Clarify を使用するための準備（Explaining Predictions）
ここではAmazon SageMaker Clarifyの「特徴量分析（どの特徴量が予測に影響を与えたか（寄与率））」の機能を使用してみます。
ビジネスニーズや法規制の拡大により、モデルがなぜそのような決定を下したのかの説明が求められることがあります。SageMaker Clarify は、SHAPという指標を使用して、各入力特徴量が最終的な決定にどのような貢献をしているかを説明します。

In [ ]:
# 必要な諸設定
# 詳細はこちら https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-explainability.html

from sagemaker import clarify
from datetime import datetime

sess = sagemaker.Session()

shap_config = clarify.SHAPConfig(
    baseline=[test_data.drop(['y_no', 'y_yes'], axis=1).iloc[0].values.tolist()],
    num_samples=15,
    agg_method="mean_abs",
    save_local_shap_values=True,
)

explainability_output_path = "s3://{}/{}/clarify-explainability".format(
    bucket, "sagemaker/DEMO-sagemaker-clarify"
)

explainability_data_config = clarify.DataConfig(
    # s3_data_input_path=train_uri,
    s3_data_input_path='s3://{}/{}/train'.format(bucket, prefix),
    s3_output_path=explainability_output_path,
    label="y_yes",
    headers=train_df.columns.to_list(),
    dataset_type="text/csv",
)

model_name = "DEMO-clarify-model-{}".format(datetime.now().strftime("%d-%m-%Y-%H-%M-%S"))
model_config = clarify.ModelConfig(
    model_name = model_name,
    instance_type="ml.m4.xlarge",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv",
)

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, instance_count=1, instance_type="ml.m4.xlarge", sagemaker_session=sess
)

In [ ]:
# モデルの登録
model = xgb.create_model(name=model_name)
container_def = model.prepare_container_def()
sess.create_model(model_name, role, container_def)

### モデルのトレーニング with SageMaker Experiments
SageMaker Experimentsを使用するには、必要なモジュールをインポートした後`with Run()...`を記載するだけです。

In [ ]:
from sagemaker.experiments.run import Run
import time

experiment_name="direct-marketing-sample"

# SageMaker Experimentsに記録したい学習の内容を "with Run..." で囲む（だけ）
with Run(experiment_name=experiment_name, sagemaker_session=sess) as run:

    # モデルの学習
    xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})
    
    #　SageMaker Clarify
    clarify_processor.run_explainability(data_config=explainability_data_config, model_config=model_config, explainability_config=shap_config,)

モデルの学習が完了したら、SageMaker StudioのタブからExperimentsにアクセスしてください。
モデルの学習に関する情報や、モデルの説明性に関する分析結果を確認することができます。

--
## Appendix1_Feature_Engineering

### 特徴量エンジニアリングの例：相関分析

In [ ]:
# cell 07
for column in data.select_dtypes(include=['object']).columns:
    if column != 'y':
        print(pd.crosstab(index=data[column], columns=data['y'], normalize='columns'))

for column in data.select_dtypes(exclude=['object']).columns:
    print(column)
    hist = data[[column, 'y']].hist(by='y', bins=30)
    plt.show()

Notice that:

* Customers who are-- "blue-collar", "married", "unknown" default status, contacted by "telephone", and/or in "may" are a substantially lower portion of "yes" than "no" for subscribing.
* Distributions for numeric variables are different across "yes" and "no" subscribing groups, but the relationships may not be straightforward or obvious.

Now let's look at how our features relate to one another.

In [ ]:
# cell 08
print(data.corr())
pd.plotting.scatter_matrix(data, figsize=(12, 12))
plt.show()

Notice that:
* Features vary widely in their relationship with one another.  Some with highly negative correlation, others with highly positive correlation.
* Relationships between features is non-linear and discrete in many cases.

--
## Appendix2_Automatic_Model_Tuning(Optional)
Amazon SageMaker の自動モデルチューニングは、ハイパーパラメータチューニング（HPO）としても知られており、指定したアルゴリズムとハイパーパラメータの範囲を使ってデータセット上で多くのトレーニングジョブを実行することで、モデルの最適なバージョンを見つけます。そして、あなたが選択した指標で測定した結果、最高のパフォーマンスを発揮するモデルになるハイパーパラメータ値を選択します。
例えば、このマーケティング・データセットでバイナリ分類問題を解きたいとします。あなたのゴールは、XGBoost アルゴリズムのモデルをトレーニングすることで、アルゴリズムの曲線下面積（auc）メトリックを最大化することです。最適なモデルを訓練するために、eta、alpha、min_child_weight、max_depthのハイパーパラメータのどの値を使用すればよいかはわかりません。これらのハイパーパラメータの最適な値を見つけるために、Amazon SageMaker のハイパーパラメータチューニングが、選択した目的指標によって測定される最適なトレーニングジョブになる値の組み合わせを見つけるために検索する値の範囲を指定することができます。ハイパーパラメータチューニングは、指定した範囲のハイパーパラメータ値を使用するトレーニングジョブを起動し、最も高い Auc を持つトレーニングジョブを返します。


In [ ]:
# cell 22
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                            'min_child_weight': ContinuousParameter(1, 10),
                            'alpha': ContinuousParameter(0, 2),
                            'max_depth': IntegerParameter(1, 10)}


In [ ]:
# cell 23
objective_metric_name = 'validation:auc'

In [ ]:
# cell 24
tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=20,
                            max_parallel_jobs=3)


In [ ]:
# cell 25
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

In [ ]:
# cell 26
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

In [ ]:
# cell 27
# return the best training job name
tuner.best_training_job()

In [ ]:
# cell 28
#  Deploy the best trained or user specified model to an Amazon SageMaker endpoint
tuner_predictor = tuner.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

In [ ]:
# cell 29
# Create a serializer
tuner_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [ ]:
# cell 30
# Predict
predictions = predict(test_data.drop(['y_no', 'y_yes'], axis=1).to_numpy(),tuner_predictor)

In [ ]:
# cell 31
# Collect predictions and convert from the CSV output our model provides into a NumPy array
pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

In [ ]:
# cell 33
tuner_predictor.delete_endpoint(delete_endpoint_config=True)

--
## Appendix3_Serverless_Deployment(Optional)
After training the model, retrieve the model artifacts so that we can deploy the model to an endpoint.

In [ ]:
# Setup clients
import boto3

client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")

In [ ]:
# Retrieve model data from training job
model_artifacts = xgb.model_data
model_artifacts

### Model Creation
Create a model by providing your model artifacts, the container image URI, environment variables for the container (if applicable), a model name, and the SageMaker IAM role.

In [ ]:
from time import gmtime, strftime

model_name = "xgboost-serverless" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)

# dummy environment variables
byo_container_env_vars = {"SAGEMAKER_CONTAINER_LOG_LEVEL": "20", "SOME_ENV_VAR": "myEnvVar"}

create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": container,
            "Mode": "SingleModel",
            "ModelDataUrl": model_artifacts,
            "Environment": byo_container_env_vars,
        }
    ],
    ExecutionRoleArn=role,
)

print("Model Arn: " + create_model_response["ModelArn"])

### Endpoint Configuration Creation
This is where you can adjust the Serverless Configuration for your endpoint. The current max concurrent invocations for a single endpoint, known as MaxConcurrency, can be any value from 1 to 200, and MemorySize can be any of the following: 1024 MB, 2048 MB, 3072 MB, 4096 MB, 5120 MB, or 6144 MB.

In [ ]:
xgboost_epc_name = "xgboost-serverless-epc" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=xgboost_epc_name,
    ProductionVariants=[
        {
            "VariantName": "byoVariant",
            "ModelName": model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 4096,
                "MaxConcurrency": 1,
            },
        },
    ],
)

print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

### Serverless Endpoint Creation
Now that we have an endpoint configuration, we can create a serverless endpoint and deploy our model to it. When creating the endpoint, provide the name of your endpoint configuration and a name for the new endpoint.

In [ ]:
endpoint_name = "xgboost-serverless-ep" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=xgboost_epc_name,
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Wait until the endpoint status is InService before invoking the endpoint.

In [ ]:
# wait for endpoint to reach a terminal state (InService) using describe endpoint
import time

describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

describe_endpoint_response

### Endpoint Invocation
Invoke the endpoint by sending a request to it. The following is a sample data point grabbed from the CSV file downloaded from the Direct Marketing dataset.

In [ ]:
payload = b"3., 999.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 1.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0., 0.,   0.,   0.,   0.,   0.,   1.,   0.,   1.,   0.,   0.,   1., 0.,   0.,   1.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   1., 0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0., 0.,   1.,   0."

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType="text/csv",
)

print(response["Body"].read())

### Clean Up
Delete any resources you created in this notebook that you no longer wish to use.

In [ ]:
client.delete_model(ModelName=model_name)
client.delete_endpoint_config(EndpointConfigName=xgboost_epc_name)
client.delete_endpoint(EndpointName=endpoint_name)

---

## Extensions

このサンプルでは、比較的小さなデータセットを分析しましたが、分散管理されたトレーニングやリアルタイムモデルホスティングといった Amazon SageMaker の機能を利用しています。 予測精度をさらに向上させるために、予測値のしきい値を微調整して、偽陽性と偽陰性の組み合わせを変えたり、ハイパーパラメータチューニングのようなテクニックを探求することもできます。 実世界のシナリオでは、手作業で特徴量を設計するのにもっと時間をかけるでしょうし、最初のデータセットでは得られなかった顧客情報を含む追加データセットを探すことになるでしょう。